<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Deep Learning Para Aplicações de Inteligência Artificial com Python e C++</font>
## <font color='blue'>Projeto 5</font>
### <font color='blue'>Deep Learning Para Detecção de Fraudes em Transações Financeiras com Criptomoedas</font>

## Instalando e Carregando os Pacotes

In [1]:
!pip install -q -U watermark

In [2]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [3]:
# Imports
import sklearn
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import Input
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [4]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando e Compreendendos Dados

In [5]:
# Carrega os dados
df_dsa = pd.read_csv('dataset.csv')

In [6]:
# Shape
df_dsa.shape

(9841, 51)

In [7]:
# Visualiza as primeiras linhas
df_dsa.head()

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [8]:
# Variável alvo
df_dsa.FLAG.value_counts()

FLAG
0    7662
1    2179
Name: count, dtype: int64

In [9]:
df_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 51 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            9841 non-null   int64  
 1   Index                                                 9841 non-null   int64  
 2   Address                                               9841 non-null   object 
 3   FLAG                                                  9841 non-null   int64  
 4   Avg min between sent tnx                              9841 non-null   float64
 5   Avg min between received tnx                          9841 non-null   float64
 6   Time Diff between first and last (Mins)               9841 non-null   float64
 7   Sent tnx                                              9841 non-null   int64  
 8   Received Tnx                                          9841

## Limpeza dos Dados

In [10]:
# Visualiza as primeiras linhas
df_dsa.head()

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [11]:
# Ajusta o nome para minúsculo
df_dsa.columns = [x.lower() for x in df_dsa.columns]

In [12]:
# Essas colunas não serão relevantes para análise
cols_to_drop = [' erc20 most sent token type',
                ' erc20_most_rec_token_type',
                'address',
                'index',
                'unnamed: 0']

In [13]:
# Seleciona os atributos filtrando as colunas que serão removidas e a variável alvo
atributos = [x for x in df_dsa.columns if (x != 'flag' and x not in cols_to_drop)]

In [14]:
atributos

['avg min between sent tnx',
 'avg min between received tnx',
 'time diff between first and last (mins)',
 'sent tnx',
 'received tnx',
 'number of created contracts',
 'unique received from addresses',
 'unique sent to addresses',
 'min value received',
 'max value received ',
 'avg val received',
 'min val sent',
 'max val sent',
 'avg val sent',
 'min value sent to contract',
 'max val sent to contract',
 'avg value sent to contract',
 'total transactions (including tnx to create contract',
 'total ether sent',
 'total ether received',
 'total ether sent contracts',
 'total ether balance',
 ' total erc20 tnxs',
 ' erc20 total ether received',
 ' erc20 total ether sent',
 ' erc20 total ether sent contract',
 ' erc20 uniq sent addr',
 ' erc20 uniq rec addr',
 ' erc20 uniq sent addr.1',
 ' erc20 uniq rec contract addr',
 ' erc20 avg time between sent tnx',
 ' erc20 avg time between rec tnx',
 ' erc20 avg time between rec 2 tnx',
 ' erc20 avg time between contract tnx',
 ' erc20 min val

In [15]:
# Extrai valores únicos
valores_unicos = df_dsa.nunique()

In [16]:
valores_unicos

unnamed: 0                                              9841
index                                                   4729
address                                                 9816
flag                                                       2
avg min between sent tnx                                5013
avg min between received tnx                            6223
time diff between first and last (mins)                 7810
sent tnx                                                 641
received tnx                                             727
number of created contracts                               20
unique received from addresses                           256
unique sent to addresses                                 258
min value received                                      4589
max value received                                      6302
avg val received                                        6767
min val sent                                            4719
max val sent            

In [17]:
# Mantém somente atributos com mais de um valor único (atributos que não são constantes)
atributos = [x for x in atributos if x in valores_unicos.loc[(valores_unicos > 1)]]

In [18]:
df_dsa[atributos].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 38 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   avg min between sent tnx                              9841 non-null   float64
 1   avg min between received tnx                          9841 non-null   float64
 2   time diff between first and last (mins)               9841 non-null   float64
 3   sent tnx                                              9841 non-null   int64  
 4   received tnx                                          9841 non-null   int64  
 5   number of created contracts                           9841 non-null   int64  
 6   unique received from addresses                        9841 non-null   int64  
 7   unique sent to addresses                              9841 non-null   int64  
 8   min value received                                    9841

## Criação do Pipeline de Pré-Processamento

**BaseEstimator**: Esta é a classe base para todos os estimadores no scikit-learn. Um estimador é qualquer objeto que pode estimar alguns parâmetros baseados em um conjunto de dados. Por exemplo, um algoritmo de Machine Learning como uma regressão linear ou uma árvore de decisão é um estimador. A classe BaseEstimator fornece funcionalidades básicas, como métodos para configurar os parâmetros de um estimador e para obter esses parâmetros com get_params() e set_params(). 

**TransformerMixin**: Esta é uma classe mista (mixin) usada para adicionar funcionalidade de transformação a um estimador. No scikit-learn, um "transformador" é um tipo de estimador que pode transformar um conjunto de dados. Por exemplo, ele pode ser usado para normalizar ou padronizar dados, selecionar ou extrair características, etc. A classe TransformerMixin adiciona o método fit_transform(), que é um método conveniente que primeiro ajusta o transformador ao conjunto de dados (usando fit()) e então aplica a transformação ao mesmo conjunto de dados (usando transform()). Isso é útil para evitar ter que chamar fit() e transform() separadamente durante o pré-processamento dos dados.

A combinação dessas duas classes é frequentemente usada ao criar um estimador personalizado ou transformador com o scikit-learn. Ao herdar dessas classes, você garante que seu estimador personalizado se integre bem com outras funcionalidades de scikit-learn, como pipelines e outras ferramentas de modelagem.

In [19]:
# Definição de uma classe personalizada que herda de BaseEstimator e TransformerMixin
class DSAPipeSteps(BaseEstimator, TransformerMixin):

    # Método construtor para inicializar a classe com uma lista de colunas
    def __init__(self, columns=[]):
        
        # Atribuição do argumento columns ao atributo de instância self.columns
        self.columns = columns

    # Método fit usado para ajustar (treinar) a transformação nos dados de treinamento
    def fit(self, X, y = None):
        
        # Retorna a própria instância, indicando que o método não faz modificações
        return self

    # Método transform para transformar os dados de entrada
    def transform(self, X):
        
        # Faz uma cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        
        # Retorna os dados sem modificações
        return X

In [20]:
# Definição de uma classe que herda de DSAPipeSteps
class DSASelecionaColunas(DSAPipeSteps):

    # Método transform para transformar os dados de entrada
    def transform(self, X):
        
        # Faz uma cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        
        # Seleciona e retorna apenas as colunas especificadas em self.columns
        return X[self.columns]

In [21]:
# Definição de uma classe que herda de DSAPipeSteps
class DSAPreencheDados(DSAPipeSteps):

    # Método fit para ajustar a transformação nos dados de treinamento
    def fit(self, X, y = None):
        
        # Calcula a média de cada coluna especificada em self.columns e armazena no dicionário self.means
        self.means = { col: X[col].mean() for col in self.columns }
        
        # Retorna a própria instância, indicando que o método não faz modificações
        return self

    # Método transform para transformar os dados de entrada
    def transform(self, X):
        
        # Faz uma cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        
        # Itera sobre cada coluna especificada em self.columns
        for col in self.columns:
            
            # Preenche valores ausentes na coluna com a média calculada na fase de ajuste
            X[col] = X[col].fillna(self.means[col])
        
        # Retorna os dados transformados
        return X

In [22]:
# Definição de uma classe que herda de DSAPipeSteps
class DSAPadronizaDados(DSAPipeSteps):

    # Método fit para ajustar o scaler nos dados de treinamento
    def fit(self, X, y = None):
        
        # Inicializa uma instância de StandardScaler para padronizar os dados
        self.scaler = StandardScaler()
        
        # Ajusta o scaler nas colunas especificadas em self.columns
        self.scaler.fit(X[self.columns])
        
        # Retorna a própria instância, indicando que o método não faz modificações
        return self

    # Método transform para transformar os dados de entrada
    def transform(self, X):
        
        # Faz uma cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        
        # Aplica a transformação de padronização nas colunas especificadas
        X[self.columns] = self.scaler.transform(X[self.columns])
        
        # Retorna os dados transformados
        return X

In [23]:
# Definição de uma classe que herda de DSAPipeSteps
class DSAObtemDados(DSAPipeSteps):

    # Método transform para transformar os dados de entrada
    def transform(self, X):
        
        # Faz uma cópia dos dados de entrada para evitar alterar os dados originais
        X = X.copy()
        
        # Retorna os valores do DataFrame como um array NumPy
        return X.values

Pipeline no scikit-learn é uma ferramenta que encadeia várias etapas de transformação e um estimador final em um único objeto. Isso permite que você configure um processo de Machine Learning que inclui etapas de pré-processamento (como normalização, padronização e extração de características) seguidas pela aplicação de um modelo de aprendizado de máquina. Cada etapa da pipeline é representada por uma tupla contendo um nome e um objeto de transformação ou modelo.

In [24]:
# Cria o pipeline
dsa_pipe_prepropcessamento = Pipeline([('feature_selection', DSASelecionaColunas(atributos)),
                                       ('fill_missing', DSAPreencheDados(atributos)),
                                       ('standard_scaling', DSAPadronizaDados(atributos)),
                                       ('returnValues', DSAObtemDados())])

In [25]:
# Variáveis de entrada
X = df_dsa[atributos]

In [26]:
# Variável de saída
y = df_dsa['flag']

In [27]:
# Ajusta o tipo da variável de saída
y = to_categorical(y)

In [28]:
# Divide os dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [29]:
# Padroniza os dados
X_treino = dsa_pipe_prepropcessamento.fit_transform(X_treino)
X_teste = dsa_pipe_prepropcessamento.transform(X_teste)

In [30]:
X_treino

array([[-0.23259911, -0.35751318, -0.67960935, ..., -0.01331307,
        -0.21172682, -0.2313969 ],
       [-0.23259911,  0.0138933 , -0.06181513, ..., -0.01331307,
        -0.21172682, -0.17365057],
       [-0.23259911, -0.34333968, -0.67254633, ..., -0.01331307,
        -0.21172682, -0.2313969 ],
       ...,
       [-0.23259911, -0.35751318, -0.67960389, ..., -0.01331307,
        -0.21172682, -0.28914324],
       [-0.22473065, -0.35751318, -0.67858957, ..., -0.01331307,
        -0.21172682, -0.28914324],
       [-0.23250192, -0.35750828, -0.67958981, ..., -0.01331307,
        -0.21172682, -0.28914324]])

In [31]:
X_teste

array([[-1.54379550e-01, -2.63825638e-01,  3.32086109e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -4.11559255e-04],
       [-9.86466733e-02, -3.20183744e-01, -6.49312398e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -2.31396905e-01],
       [-2.25103708e-01, -3.57513182e-01, -6.78637885e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -2.89143241e-01],
       ...,
       [-2.32514779e-01, -3.57494918e-01, -6.79590458e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -2.89143241e-01],
       [-2.32599109e-01, -3.57233873e-01, -6.75564107e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -2.31396905e-01],
       [-2.31476133e-01, -3.57513182e-01, -6.79391008e-01, ...,
        -1.33130727e-02, -2.11726819e-01, -2.89143241e-01]])

## Construção do Modelo de Deep Learning

In [32]:
# Cria sequência de camadas
modelo_dsa = Sequential()

In [33]:
# Adiciona uma camada de entrada ao modelo com a forma especificada pelo comprimento de 'atributos'
modelo_dsa.add(Input(shape = (len(atributos),)))

# Adiciona uma camada densa ao modelo com 'len(atributos)' unidades e a função de ativação 'relu'
modelo_dsa.add(Dense(len(atributos), activation = 'relu'))

# Adiciona uma camada densa ao modelo com 20 unidades e a função de ativação 'relu'
modelo_dsa.add(Dense(20, activation = 'relu'))

# Adiciona uma camada densa ao modelo com 5 unidades e a função de ativação 'relu'
modelo_dsa.add(Dense(5, activation = 'relu'))

# Adiciona uma camada densa ao modelo com 2 unidades e a função de ativação 'softmax' (camada de saída)
modelo_dsa.add(Dense(2, activation = 'softmax'))

In [34]:
# Compilação do modelo
modelo_dsa.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [35]:
# Sumário
modelo_dsa.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 38)             │         1,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,379 (9.29 KB)

 Trainable params: 2,379 (9.29 KB)

 Non-trainable params: 0 (0.00 B)

## Treinamento e Avaliação do Modelo

In [36]:
%%time
modelo_dsa.fit(X_treino, y_treino, validation_data = (X_teste, y_teste), epochs = 10)

Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.7175 - loss: 0.5995 - val_accuracy: 0.8490 - val_loss: 0.3920
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8412 - loss: 0.3807 - val_accuracy: 0.8557 - val_loss: 0.3119
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.8529 - loss: 0.3081 - val_accuracy: 0.8612 - val_loss: 0.2817
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.8892 - loss: 0.2563 - val_accuracy: 0.9309 - val_loss: 0.2569
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.9146 - loss: 0.2359 - val_accuracy: 0.9279 - val_loss: 0.2304
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.9329 - loss: 0.2105 - val_accuracy: 0.9370 - val_loss: 0.2065
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.9394 - loss: 0.1794 - val_accuracy: 0.9421 - val_loss: 0.2013
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.9427 - loss: 0.1696 - 

In [37]:
# Previsões com dados de teste
previsoes_teste = [np.argmax(x) for x in modelo_dsa.predict(X_teste)]

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 266us/step


In [38]:
# Calcula a Acurácia
acc = metrics.accuracy_score(previsoes_teste, [np.argmax(y) for y in y_teste])

A acurácia é uma métrica de avaliação de modelos de classificação. Ela representa a proporção de previsões corretas feitas pelo modelo em relação ao total de amostras avaliadas. Em outras palavras, é a quantidade de previsões corretas dividida pelo total de previsões feitas, indicando o quão bem o modelo está classificando as amostras corretamente.

In [39]:
print(f'Acurácia nos Dados de Teste: {acc:,.2%}')

Acurácia nos Dados de Teste: 94.92%


In [40]:
# Calcula AUC
auc = metrics.roc_auc_score([np.argmax(y) for y in y_teste], modelo_dsa.predict(X_teste)[:,1])

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 187us/step


A Área Sob a Curva (AUC, na sigla em inglês) é uma métrica usada para avaliar o desempenho de modelos de classificação binária. Ela é baseada na curva ROC (Receiver Operating Characteristic), que é um gráfico que mostra a relação entre a taxa de verdadeiros positivos (sensibilidade) e a taxa de falsos positivos (1 - especificidade) para diferentes limiares de decisão.

A AUC representa a área total sob a curva ROC e pode variar de 0 a 1. Uma AUC de 0,5 indica um desempenho aleatório, enquanto uma AUC de 1,0 indica uma classificação perfeita. Portanto, quanto mais próximo de 1 for o valor da AUC, melhor será a capacidade do modelo de distinguir entre as classes positivas e negativas.

In [41]:
print(f'AUC nos Dados de Teste - {auc:,.2%}')

AUC nos Dados de Teste - 97.53%


## Deploy do Modelo e Detecção de Fraudes em Novas Transações com Criptomoedas

In [42]:
# Carrega os novos dados de uma transação
novos_dados = pd.read_csv('novos_dados.csv')

In [43]:
# Dados no formato original
novos_dados

,avg min between sent tnx,avg min between received tnx,time diff between first and last (mins),sent tnx,received tnx,number of created contracts,unique received from addresses,unique sent to addresses,min value received,max value received,...,erc20 uniq sent addr.1,erc20 uniq rec contract addr,erc20 min val rec,erc20 max val rec,erc20 avg val rec,erc20 min val sent,erc20 max val sent,erc20 avg val sent,erc20 uniq sent token name,erc20 uniq rec token name
0,2570.59,3336.01,30572.7,8,3,0,2,4,0.1,40.0,...,0.0,1.0,600.0,600.0,600.0,0.0,0.0,0.0,0.0,1.0


In [44]:
# Aplica o mesmo pipeline aplicado aos dados de treino
novos_dados_transformados = dsa_pipe_prepropcessamento.transform(novos_dados)

In [45]:
# Dados no formato que o modelo espera receber
novos_dados_transformados

array([[-0.11012514, -0.20890417, -0.5852175 , -0.14846488, -0.17435227,
        -0.02949086, -0.10071957, -0.08219271, -0.12770293, -0.03945017,
        -0.0281488 , -0.04686515, -0.05420181, -0.110975  ,  0.        ,
        -0.01204994, -0.01204994, -0.20568049, -0.02975031, -0.03630726,
        -0.01204994, -0.0197842 , -0.08228942, -0.06411176, -0.01425745,
        -0.02258837, -0.05367989, -0.09036273, -0.05319145, -0.22783703,
         0.00627018, -0.05510854, -0.02401607, -0.01364824, -0.0135741 ,
        -0.01331307, -0.21172682, -0.2313969 ]])

In [46]:
# Extrai a previsão de maior probabilidade
previsao = [np.argmax(x) for x in modelo_dsa.predict(novos_dados_transformados)]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [47]:
type(previsao)

list

In [48]:
# Resultado
if previsao[0] == 0:
    print("Segundo o modelo, essa transação não representa uma Fraude.")
else:
    print("Segundo o modelo, essa transação pode representar uma Fraude. Acione a verificação humana!")

Segundo o modelo, essa transação não representa uma Fraude.


In [49]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [50]:
#%watermark -v -m

In [51]:
#%watermark --iversions

# Fim